# U-Net论文复现

UNet 于 2015 年在医学图像分割领域首次提出，凭借其在生物医学图像分割挑战中的优异表现（如 ISBI 挑战赛中的出色成绩）获得广泛关注。其核心思想是通过对称的编码器-解码器结构与跳跃连接（skip connections）相结合，既提取深层语义特征，又保留高分辨率的空间信息，从而实现精确的像素级分割。

---

## U-Net的特点

<img src="imgs/UNet-Structure.jpeg">

### **U-Net 的结构特点**  
1. **对称的编码器-解码器结构**：  
   - **编码器（下采样路径）**：通过卷积层和池化层逐步提取输入图像的高级特征。  
   - **解码器（上采样路径）**：使用反卷积（转置卷积）或上采样操作逐步恢复图像的空间分辨率。  

2. **跳跃连接 (Skip Connections)**：  
   - 在相同层级的编码器和解码器之间建立直接连接，将低层的高分辨率特征与高层的语义特征相结合，提高分割精度并缓解梯度消失问题。

3. **无全连接层**：  
   - 使网络能够处理任意大小的输入图像，同时减少参数量，提升训练效率。  

### **U-Net 的优势**  
- **高精度分割**：跳跃连接保留了空间细节信息，提高了分割的精度，尤其适合边界清晰的对象分割。  
- **高效的数据利用率**：在小样本数据集上也能取得良好效果，适用于医学图像等数据稀缺场景。  
- **端到端训练**：整个网络可通过反向传播一次性端到端训练，无需复杂的后处理步骤。  
- **实时分割能力**：推理速度较快，适合需要实时分割的任务。  

### **U-Net 的劣势**  
- **内存占用大**：跳跃连接会导致中间特征图的大量存储，增加显存需求。  
- **对大尺寸输入不友好**：大输入图像会显著增加计算和存储开销。  
- **细粒度结构恢复有限**：对于极其复杂或细粒度的分割任务，单纯的跳跃连接可能不足以完全恢复细节。  
- **上采样可能引入伪影**：如果上采样设计不当，容易在分割结果中引入伪影或失真。  

### **U-Net 的架构**  
1. **编码器部分**（下采样路径）：  
   - 每个下采样模块包含两个 3x3 卷积（ReLU 激活）+ 一个 2x2 最大池化（stride=2）。  
   - 每下采样一次，特征通道数翻倍（如从 64 → 128 → 256 → ...）。  

2. **瓶颈层**：  
   - 位于编码器和解码器之间，由两个 3x3 卷积层组成，用于捕捉全局上下文信息。  

3. **解码器部分**（上采样路径）：  
   - 每个上采样模块包含一个 2x2 上采样（或反卷积），特征通道数减半。  
   - 上采样后的特征图与对应的编码器特征图进行拼接（concatenation），再通过两个 3x3 卷积层提炼特征。  

4. **输出层**：  
   - 使用一个 1x1 卷积，将多通道特征映射到所需的类别数，实现像素级分类。  

### **U-Net 的典型应用**  
- **医学图像分割**：细胞分割、肿瘤检测、器官定位（如 MRI、CT 图像分割）。  
- **遥感图像分割**：土地覆盖分类、建筑物和道路提取。  
- **自动驾驶**：道路、车道线和障碍物分割。  
- **工业缺陷检测**：对生产中的产品进行缺陷分割与检测。  
- **生物图像分析**：显微镜图像中的组织和细胞结构识别。  
- **文本行分割**：用于文档图像的文本行和段落分割。  

---


## ResNet的代码实现
### 1. 必要库的导入

In [3]:
import torch
from torch import nn,optim
import numpy as np
import matplotlib.pyplot as plt
from torchvision import datasets,transforms
from torchsummary import summary
from torch.utils.data import DataLoader
%matplotlib inline

device = torch.device("cuda" if torch.cuda.is_available()
                      else "mps" if torch.mps.is_available()
                      else "cpu")
print(torch.__version__)
print(device)


2.6.0
mps
